# Monarq Device

Ce document a pour objectif d'expliquer ce dont vous avez besoin pour utiliser MonarqDevice.

Ce "device" vous permet de communiquer directement avec MonarQ. MonarqDevice est aussi équipé de transpileur fait par Calcul Québec, qui est optimisé pour MonarQ.


## Usage typique

Voici la manière typique d'utiliser le device ```monarq.default``` :

0. Tout d'abord, assurez-vous d'avoir la version Python ```3.11.X``` d'installée.

1. Installez le plugin (vous pouvez vous référer au README)

2. Importez les dépendances :

In [1]:
# Cette ligne importe la librairie PennyLane. Elle contient tout ce dont vous avez besoin pour la programmation quantique.
import pennylane as qml

# Cette ligne importe la classe client, nécessaire pour s'authentifier sur MonarQ
from pennylane_calculquebec.API.client import CalculQuebecClient

3. Créez un `Client` pour votre device. Pour plus de détails sur l'utilisation de vos informations d'authentification et de l'identifiant de projet, visitez [cette page](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_client.ipynb)

In [2]:
# Voilà comment créer un client. Changez les valeurs dans la parenthèse pour vos identifiants
my_client = CalculQuebecClient(
    host="your host",
    user="your user",
    access_token="your access token",
    project_id="your project id",  # plus d'information sur ce paramètre dans using_client.ipynb
)

4. Créez un device en utilisant votre objet client

Il y a 3 arguments obligatoires pour le device : 
- le nom ```monarq.default```
- le client
- le nombre d'échantillons

Il y a 2 arguments optionnels : 
- Le nombre de qubits (un entier), ou les qubits exacts (un tableau)
- Une [configuration](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_configurations.ipynb)

In [3]:
# Vous pouvez définir le nombre de qubits (cela créera un ensemble de qubits de 0 à n exclusivement)
dev = qml.device(
    "monarq.default",
    client=my_client,
    wires=3,  # celà définira les qubits [0, 1, 2], soit 3 qubits
)

# Vous pouvez aussi définir quels qubits exactement devront être utilisés
dev = qml.device(
    "monarq.default",
    client=my_client,
    wires=[4, 1, 5],  # celà définira les qubits [4, 1, 5]
)

# Vous n'êtes pas obligé de spécifier de qubits. Le device se fiera alors aux qubits utilisés par les opérations et mesures de votre circuit
dev = qml.device(
    "monarq.default",
    client=my_client,
)

Il faut noter que les qubits que vous spécifiez pour votre device et vos opérations ne seront pas nécessairement utilisés sur MonarQ. Cela est dû aux phases de placement et de routage du transpileur. 

Pour plus d'information, lisez [using transpiler](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_transpiler.ipynb)

5. Créez votre circuit

In [4]:
# Cette ligne indique que la fonction qui suit est une fonction quantique (un qnode)
@qml.set_shots(1000)
@qml.qnode(dev)
# Ceci est votre fonction quantique. Elle sera utilisée plus tard pour exécuter votre circuit
def circuit():
    # Ceci est une porte d'Hadamard. Elle met le qubit en superposition.
    qml.Hadamard(wires=0)
    # Ceci est une porte CNOT. Elle connecte les qubits 0 et 1.
    qml.CNOT(wires=[0, 1])
    # Ceci est aussi une porte CNOT, mais elle agit sur les qubits 1 et 2.
    qml.CNOT(wires=[1, 2])

    # Cette instruction effectuera une mesure sur tous les qubits du circuits (trois qubits, dans ce cas).
    # Vous pouvez aussi définir sur quels qubits vous voulez effectuer la mesure, en utilisant l'argument "wires"
    # return qml.counts(wires=[0, 1]) # Ceci appliquerait une mesure sur les fils 0 et 1.
    return qml.counts()


# Cette ligne montre votre circuit avant la compilation
result = qml.draw(circuit, level="device")()
print(result)

0: ──X90────╭●──────────────┤ ╭Counts
4: ──X90──Z─╰Z──X90─╭●──────┤ ├Counts
8: ──X90──Z─────────╰Z──X90─┤ ╰Counts


Pour plus d'information à propos des opérations en PennyLane, cliquez [ici](https://docs.pennylane.ai/en/stable/introduction/operations.html)

6. Exécutez le circuit et utilisez les résultats comme vous le désirez

In [5]:
# Cette ligne exécute le circuit et retourne les résultats, les plaçant dans la variable "results"
results = circuit()
# Cette ligne montre le résultat entant que dictionnaire.
print(results)

{'000': array(480), '001': array(24), '010': array(6), '011': array(37), '100': array(38), '101': array(43), '110': array(39), '111': array(333)}


## Plus d'information

- [clients](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_client.ipynb)
- [configurations](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_configurations.ipynb)
- [transpileur](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_transpiler.ipynb)
- [étapes de processing](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_custom_steps.ipynb)
- [portes customs](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_custom_gates.ipynb)
- [api adapter](https://github.com/calculquebec/pennylane-calculquebec/blob/main/doc/for_developers/using_api_adapter.ipynb)